# Main Code

In [45]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.request
from urllib.parse import urlparse
import time
import os
import re
import urllib3

# declares timer and locks absolute path to working directory
before = time.perf_counter()
absolute_path = os.getcwd()

# solves SSL certificate issues error when retrieving files 
import os, ssl
if (not os.environ.get("PYTHONHTTPSVERIFY", "") and getattr(ssl, "_create_unverified_context", None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

# Solving SSL certificate issue
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# retrieve links from site
url = "https://www.junaeb.cl/ive"
page = requests.get(url, verify=False)
soup = BeautifulSoup(page.text, "lxml")
a_tags = soup.find_all('a')
all_links = [link.get('href') for link in a_tags]
links = []
for link in all_links:
    if ".xls" in link and "IVESINAE_COMUNA_2013" not in link:
        links.append(link)
        
years_df = []

# creates data_temp folder and changes working directory
if os.path.isdir("data_temp") == False:
    os.mkdir("data_temp")    
os.chdir("data_temp")

# downloads files
for url in links:
    encoded_url = urllib.parse.quote(url.encode('utf-8'),':/')
    filename = encoded_url[encoded_url.rfind("/")+1:]
    if filename not in os.listdir():
        urllib.request.urlretrieve(encoded_url, filename)
        print("{:.2f} s | Downloaded {}".format(time.perf_counter()-before, url))
    
# processing function
def clean(file, year):
    
    #defines scope of variable
    global years_df
        
    # reads Excel file and defines available sheets on file
    try:
        df = pd.read_excel(file, sheet_name = ["BASICA", "MEDIA"], encoding="utf-8")
        sheets = ["BASICA", "MEDIA"]
    except:
        df = pd.read_excel(file, sheet_name = ["BÁSICA", "MEDIA"], encoding="utf-8")
        sheets = ["BÁSICA", "MEDIA"]
    

    for tb in sheets:
        # deletes last two columns and last row
        df[tb] = df[tb].drop(df[tb].columns[14:], axis=1)
        last_row = df[tb].shape[0]
        df[tb] = df[tb].drop(last_row-1)

        # fills NaN with 0
        df[tb].iloc[:, 9:] = df[tb].iloc[:, 9:].fillna(0)

        # creates new column to make dataframe tidy
        if tb == 'BÁSICA':
            df[tb]["level"] = pd.Series(['BASICA'] * df[tb].shape[0])
        else:
            df[tb]["level"] = pd.Series([tb] * df[tb].shape[0])
            
        # creates new year column
        df[tb]["year"] = pd.Series([year] * df[tb].shape[0])

        # melts dataframe to make it tidy
        priorities = df[tb].columns[9:14]
        df[tb] = pd.melt(df[tb], id_vars = [x for x in df[tb].columns if x not in priorities], value_vars = priorities, var_name = "priority", value_name = "total")

        # changes column names
        df[tb].columns = ["rbd", "dv_rbd", "school_name", "dependency", "area", "region_code", "province_code", "commune_code", "commune_name", "level", "year", "priority", "total"]

        # changes column types
        num_cols = ["rbd", "dv_rbd", "region_code", "province_code", "commune_code", "total"]
        df[tb][num_cols] = df[tb][num_cols].apply(pd.to_numeric, downcast="integer")
        
        # changes string columns to uppercase
        def uppercase(row, col):
            return row[col].upper()
        
        str_cols = [x for x in df[tb].columns if x not in num_cols]
        
        for column in str_cols:
            df[tb][column] = df[tb].apply(uppercase, col=column, axis=1) 
         
    # concatenates both dataframes
    df = pd.concat([df[sheets[0]], df[sheets[1]]], ignore_index=True)
    years_df.append(df)

# function to get year information
def get_year(string):
    reg = re.search("\d", string)
    first = reg.start()
    y = string[first : first + 4]
    return y 

# processes each link
for file in os.listdir():
    new_filename = file[file.rfind("/") + 1:]
    print("{:.2f} s | Cleaning {}".format(time.perf_counter()-before, new_filename))
    clean(new_filename, get_year(new_filename))

# concatenates each year's dataframe
df = pd.concat(years_df, ignore_index=True)
print("{:.2f} s | Concatenated each year's dataframe.".format(time.perf_counter()-before))

# classifies dependencies
def classify_dependencies(row, col):
    if "SUB" in row[col]:
        return "PARTICULAR SUBVENCIONADO"
    elif "CORP" in row[col]:
        return "CORPORACION MUNICIPAL"
    elif "MUNI" in row[col]:
        return "MUNICIPAL DAEM"
    elif "DELE" in row[col]:
        return "ADMINISTRACION DELEGADA"
    
df["dependency"] = df.apply(classify_dependencies, col = "dependency", axis = 1)
print("{:.2f} s | Classified dependency column.".format(time.perf_counter()-before))

# classifies priorities
def classify_priorities(row, col):
    if "1" in row[col]:
        return "PRIMERA PRIORIDAD"
    elif "2" in row[col]:
        return "SEGUNDA PRIORIDAD"
    elif "3" in row[col]:
        return "TERCERA PRIORIDAD"
    elif "INFORMACIÓN" in row[col]:
        return "SIN INFORMACION"
    elif "APLICA" in row[col]:
        return "NO VULNERABLES"
    else:
        return row[col]
    
df["priority"] = df.apply(classify_priorities, col = "priority", axis = 1)
print("{:.2f} s | Classified priority column.".format(time.perf_counter()-before))

# writes datachile official IDs for each commune and drops columns: dv_rbd and school_name
df_ids = pd.read_csv("https://raw.githubusercontent.com/datachile/datachile-etl/master/official_ids/2017_06_27_comunas_datachile_fixed.csv")
df = pd.merge(df, df_ids, left_on = "commune_code", right_on = "comuna_customs_id")
df = df[["rbd", "dependency", "area", "comuna_datachile_id", "level", "year", "priority", "total"]]

# Changes IDs and column names (dependency: administration) --> Corporación Municipal = 2
def administration(row, col):
    if "SUB" in row[col]:
        return 2
    elif "CORP" in row[col]:
        return 2
    elif "MUNI" in row[col]:
        return 1
    elif "DELE" in row[col]:
        return 4
    else:
        return np.nan
    
df["dependency"] = df.apply(administration, col = "dependency", axis = 1)
df = df.rename(columns = {"dependency":"administration"})
print("{:.2f} s | Changed administration IDs.".format(time.perf_counter()-before))

# Changes IDs and column names (area: zone)
def zone(row, col):
    if "URBANO" in row[col]:
        return 1
    elif "RURAL" in row[col]:
        return 2
    else:
        return np.nan
    
df["area"] = df.apply(zone, col = "area", axis = 1)
df = df.rename(columns = {"area":"zone_id"})
print("{:.2f} s | Changed zone IDs.".format(time.perf_counter()-before))

# Changes IDs and column names (level: education)
def education(row, col):
    if "BASICA" in row[col]:
        return 1
    elif "MEDIA" in row[col]:
        return 2
    else:
        return np.nan
    
df["level"] = df.apply(education, col = "level", axis = 1)
df = df.rename(columns = {"level":"education"})
print("{:.2f} s | Changed education IDs.".format(time.perf_counter()-before))

# Changes IDs and column names (priority)
def priorities(row, col):
    if "SIN INFORMACION" in row[col]:
        return 0
    elif "PRIMERA PRIORIDAD" in row[col]:
        return 1
    elif "SEGUNDA PRIORIDAD" in row[col]:
        return 2
    elif "TERCERA PRIORIDAD" in row[col]:
        return 3
    elif "NO VULNERABLES" in row[col]:
        return 4
    else:
        return np.nan
    
df["priority"] = df.apply(priorities, col = "priority", axis = 1)
print("{:.2f} s | Changed priority IDs.".format(time.perf_counter()-before))

# comes back to original path and exports as csv
print(df)
os.chdir(absolute_path)
df.to_csv("ive_junaeb.csv")
print("{:.2f} s | Exported csv file.".format(time.perf_counter()-before))

2.05 s | Cleaning PRIORIDADES-POR-RBD-CON-IVE-SINAE-2017-BASICA-MEDIA-COMUNA.xlsx
7.05 s | Cleaning PRIORIDADES-2016-CON-IVE-SINAE-BASICA-MEDIA-Y-COMUNAL_1.xlsx
12.07 s | Cleaning IVE-SINAE-BASICA-MEDIA-Y-COMUNAL-2014-OFICIAL-09012014.xlsx
17.20 s | Cleaning IVE-SINAE_2013-OFICIAL-14022013.xlsx
22.29 s | Cleaning PRIORIDADES-2012-B%C3%81SICA-MEDIA-COMUNA-CON-IVE-SINAE-OFICIAL.xlsx
28.41 s | Cleaning PRIORIDADES-2010-CON-IVESINAE-BASICA-MEDIA-VERSION-2-FINAL.xls
32.95 s | Cleaning PRIORIDADES-2015-CON-IVE-SINAE-BASICA-MEDIA-Y-COMUNAL.xlsx
38.00 s | Cleaning IVE-POR-RBD-BASICA-MEDIA-COMUNA-2018.xlsx
42.99 s | Cleaning PRIORIDADES-2011-BASICA-MEDIA-Y-COMUNA-CON-IVE-SINAE-OFICIAL.xls
47.81 s | Concatenated each year's dataframe.
55.55 s | Classified dependency column.
71.39 s | Classified priority column.
80.04 s | Changed administration IDs.
86.04 s | Changed zone IDs.
91.54 s | Changed education IDs.
101.34 s | Changed priority IDs.
          rbd  administration  zone_id  comuna_datachil

# <u>Optimizations and Tests</u>

### Comparison of methods to get filenames

In [6]:
%%timeit
site = "http://www.junaeb.cl/wp-content/uploads/2018/03/IVE-POR-RBD-BASICA-MEDIA-COMUNA-2018.xlsx"
cut = site[site.rfind("/")+1:]

241 ns ± 4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
%%timeit
site = "http://www.junaeb.cl/wp-content/uploads/2018/03/IVE-POR-RBD-BASICA-MEDIA-COMUNA-2018.xlsx"
cut = site.split("/")[-1]

### Function to classify dependencies

In [11]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("ive_junaeb.csv")
df["dependency"].value_counts()

TERCERA PRIORIDAD    74163
PRIMERA PRIORIDAD    74163
SEGUNDA PRIORIDAD    74163
SIN INFORMACION      74142
NO VULNERABLES       74114
NO APLICA            21464
SIN INFORMACIÓN      21436
3ª PRIORIDAD         21415
2ª PRIORIDAD         21415
1ª PRIORIDAD         21415
Name: priority, dtype: int64

In [36]:
#%%timeit
import pandas as pd

def classify(row, col):
    if "SUB" in row[col] or "Sub" in row[col]:
        return "PARTICULAR SUBVENCIONADO"
    elif "CORP" in row[col] or "Corp" in row[col]:
        return "CORPORACION MUNICIPAL"
    elif "MUNI" in row[col] or "Muni" in row[col]:
        return "MUNICIPAL DAEM"
    elif "DELE" in row[col] or "Dele" in row[col]:
        return "ADMINISTRACION DELEGADA"
    
df = pd.read_csv("ive_junaeb.csv")

df["dependency"] = df.apply(classify, col="dependency", axis=1)
df["dependency"].value_counts()


PARTICULAR SUBVENCIONADO    225585
MUNICIPAL DAEM              215025
CORPORACION MUNICIPAL        35120
ADMINISTRACION DELEGADA       2160
Name: dependency, dtype: int64

### Function to classify priorities

In [18]:
def classify_priorities(row, col):
    if "1" in row[col]:
        return "PRIMERA PRIORIDAD"
    elif "2" in row[col]:
        return "SEGUNDA PRIORIDAD"
    elif "3" in row[col]:
        return "TERCERA PRIORIDAD"
    elif "INFORMACIÓN" in row[col]:
        return "SIN INFORMACION"
    elif "APLICA" in row[col]:
        return "NO VULNERABLES"
    else:
        return row[col]

df = pd.read_csv("ive_junaeb.csv")
df["priority"] = df.apply(classify_priorities, col = "priority", axis = 1)
df["priority"].value_counts()

TERCERA PRIORIDAD    95578
PRIMERA PRIORIDAD    95578
SEGUNDA PRIORIDAD    95578
NO VULNERABLES       95578
SIN INFORMACION      95578
Name: priority, dtype: int64

### Importing official IDs

In [42]:
import pandas as pd
import numpy as np

df = pd.read_csv("ive_junaeb.csv", index_col = 0)
df_ids = pd.read_csv("https://raw.githubusercontent.com/datachile/datachile-etl/master/official_ids/2017_06_27_comunas_datachile_fixed.csv")
df = pd.merge(df, df_ids, left_on = "commune_code", right_on = "comuna_customs_id")
df = df[["rbd", "dependency", "area", "comuna_datachile_id", "level", "year", "priority", "total"]]
df.head()

print(df["dependency"].value_counts())

# dependency: administration --> Corporación Municipal = 2
def administration(row, col):
    if "SUB" in row[col]:
        return 3
    elif "CORP" in row[col]:
        return 1
    elif "MUNI" in row[col]:
        return 2
    elif "DELE" in row[col]:
        return 5
    else:
        return np.nan
    
df["dependency"] = df.apply(administration, col = "dependency", axis = 1)
df = df.rename(columns = {"dependency":"administration"})
print("{:.2f} s | Changed administration IDs.".format(time.perf_counter()-before))

# area: zone
def zone(row, col):
    if "URBANO" in row[col]:
        return 1
    elif "RURAL" in row[col]:
        return 2
    else:
        return np.nan
    
df["area"] = df.apply(zone, col = "area", axis = 1)
df = df.rename(columns = {"area":"zone_id"})
print("{:.2f} s | Changed zone IDs.".format(time.perf_counter()-before))
df["priority"].value_counts()

PARTICULAR SUBVENCIONADO    220720
MUNICIPAL DAEM              214280
CORPORACION MUNICIPAL        33570
ADMINISTRACION DELEGADA       2160
Name: dependency, dtype: int64
11353.73 s | Changed administration IDs.
11359.80 s | Changed zone IDs.


SIN INFORMACION      94146
PRIMERA PRIORIDAD    94146
TERCERA PRIORIDAD    94146
SEGUNDA PRIORIDAD    94146
NO VULNERABLES       94146
Name: priority, dtype: int64